In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import dill

#### Load the dataset

In [2]:
# Load the dataset
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Preprocess the data

In [3]:
# Drop irrevalent features
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Number of unique values in 'Geography' feature
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [5]:
# Dividing the dataset into independent and dependent features
X = df.drop(columns=['Exited'], axis=1)
y = df['Exited']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Encoding categorical features and scaling numerical features
# Setting up preprocessor
categorical_features = [feature for feature in X.columns if X[feature].dtype == 'object']
numerical_features = [feature for feature in X.columns if feature not in categorical_features]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features),
        ('scaler', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
).set_output(transform='pandas')

In [7]:
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

In [8]:
preprocessor.named_transformers_['cat'].categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [9]:
# Saving the preprocessor
with open('preprocessor.pkl', 'wb') as f:
    dill.dump(preprocessor, f)

### ANN Implementation

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

In [11]:

log_dir = "logs/fit" + datetime.now().strftime('%Y%m%d-%H%M%S')

In [12]:
# Build ANN model
def build_model(X_train_scaled, y_train, log_dir):
    model = Sequential(
        [
            Input(shape=(X_train_scaled.shape[1],)),
            Dense(64, activation='relu'), # Hidden Layer 1 connected with input layer
            Dense(32, activation='relu'), # Hidden layer 2 with HL1
            Dense(1, activation='sigmoid') # Output layer
        ]
    )
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.01),
                  loss=BinaryCrossentropy(),
                  metrics=['accuracy'])
    
    # Setup the TensorBoard
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Setup Early Stopping
    early_stopping_callback = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train_scaled,
        y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[tensorboard_callback, early_stopping_callback],
        verbose=1
    )
    return model 
    

In [13]:
model = build_model(X_train_scaled=X_train_scaled,
                    y_train=y_train,
                    log_dir=log_dir)

2025-07-13 16:13:36.597309: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-07-13 16:13:36.597345: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-13 16:13:36.597352: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-13 16:13:36.597390: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-13 16:13:36.597405: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/100


2025-07-13 16:13:36.984455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 2s 8ms/step - loss: 0.5290 - accuracy: 0.7878 - val_loss: 0.6383 - val_accuracy: 0.7969
Epoch 2/100
200/200 [==============================] - 1s 7ms/step - loss: 1.6257 - accuracy: 0.7200 - val_loss: 1.9967 - val_accuracy: 0.5844
Epoch 3/100
200/200 [==============================] - 1s 6ms/step - loss: 2.7949 - accuracy: 0.7186 - val_loss: 4.9056 - val_accuracy: 0.7406
Epoch 4/100
200/200 [==============================] - 1s 6ms/step - loss: 5.2194 - accuracy: 0.7220 - val_loss: 2.5584 - val_accuracy: 0.7825
Epoch 5/100
200/200 [==============================] - 1s 6ms/step - loss: 7.0074 - accuracy: 0.7261 - val_loss: 3.4204 - val_accuracy: 0.7638
Epoch 6/100
200/200 [==============================] - 1s 6ms/step - loss: 11.0983 - accuracy: 0.7291 - val_loss: 3.2975 - val_accuracy: 0.7781


In [14]:
model.save('model.keras')